TODO:
RETURN TOP 10 MATHCES
USE A BIGGER BOOKS DATASET

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

c:\Personal\Study Stuff\SynopsisBookSearchBackend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
#load
booksDataframe = pd.read_csv("./data/books_summary.csv")
booksDataframe = booksDataframe.drop_duplicates(subset="book_name", keep="first")

booksDataframe.to_csv("./cleaned_books.csv")

In [15]:
#load cmu dataset
column_names = [
    "Wikipedia_ID",
    "Freebase_ID",
    "Title",
    "Author",
    "Publication_Date",
    "Genres",
    "Plot_Summary"
]

# Load the dataset
booksDataframe2 = pd.read_csv(
    "./data/booksummaries.txt",
    sep="\t",
    header=None,
    names=column_names,
    dtype=str,
    na_values=["\\N"]
)


In [31]:
#process into new dataset
booksDataframe2 = booksDataframe2.rename(columns={"summary" : "summaries"})


In [32]:
#preview
print(booksDataframe2.head())

                                   book_name  \
0                                Animal Farm   
1                         A Clockwork Orange   
2                                 The Plague   
3  An Enquiry Concerning Human Understanding   
4                       A Fire Upon the Deep   

                                           summaries  
0   Old Major, the old boar on the Manor Farm, ca...  
1   Alex, a teenager living in near-future Englan...  
2   The text of The Plague is divided into five p...  
3   The argument of the Enquiry proceeds by a ser...  
4   The novel posits that space around the Milky ...  


In [35]:
#preview
test_books = booksDataframe.head(100)
booksDataframe = booksDataframe[["book_name","summaries"]]
print(booksDataframe.head())


                             book_name  \
0          The Highly Sensitive Person   
1  Why Has Nobody Told Me This Before?   
2                 The Midnight Library   
3                      Brave New World   
4                                 1984   

                                           summaries  
0   is a self-assessment guide and how-to-live te...  
1   is a collection of a clinical psychologist’s ...  
2   tells the story of Nora, a depressed woman in...  
3   presents a futuristic society engineered perf...  
4   is the story of a man questioning the system ...  


In [94]:
#merge dfs
merged_df = pd.concat([booksDataframe, booksDataframe2], ignore_index=True)
merged_df = merged_df.drop_duplicates(subset="book_name", keep="first")

In [183]:
#display merged
test_books = merged_df
print(test_books["book_name"][13434])

# Count summaries with more than 1000 words
num_long_summaries = (merged_df["summaries"].apply(lambda x: len(str(x).split())) > 300).sum()

print(f"Number of summaries with more than 2000 words: {num_long_summaries}")


The Gift of Rain
Number of summaries with more than 2000 words: 6833


In [184]:
# Filter out summaries with more than 1500 words
merged_df = merged_df[merged_df["summaries"].apply(lambda x: len(str(x).split())) <= 300]

# Reset index after removal (optional)
merged_df.reset_index(drop=True, inplace=True)

print("Entries with more than 1500 words removed.")

Entries with more than 1500 words removed.


In [185]:
merged_df.to_csv("./data/books_summaries_final.csv")

In [44]:
#load sbert
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
#create embeddings vector
summaries_embeddings = model.encode(test_books["summaries"], convert_to_numpy=True)

KeyboardInterrupt: 

In [22]:
#input
user_input = input("Describe your preferred plot c:")
user_embedding = model.encode(user_input, convert_to_tensor=True)

In [98]:
#load stuff
from transformers import AutoTokenizer
tokenizer = model.tokenizer


In [ ]:
def get_embedding(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)  # Tokenize
    max_len = 350  # Keeping space for special tokens
    overlap = 50

    
    # If text fits within limit, just encode it
    if len(tokens) <= max_len:
        return model.encode(text, convert_to_numpy=True)

    # Split into chunks
    chunks = []
    for i in range(0, len(tokens), max_len - overlap):
        chunk = tokens[i:i + max_len]  # Get chunk
        chunk_text = tokenizer.decode(chunk)  # Convert back to text
        chunks.append(chunk_text)

    # Encode each chunk separately
    chunk_embeddings = model.encode(chunks, convert_to_numpy==True)
    
    # Average embeddings to get a single vector
    return chunk_embeddings.mean(dim=0)

In [189]:
def get_synopsis_embeddings(df):

    """Compute embeddings for all book synopses and store in a list."""
    
    embeddings = [get_embedding(text) for text in df["summaries"]]
    return embeddings

In [190]:
#load embeddings
merged_df = merged_df.dropna(subset=["book_name", "summaries"])
merged_df.to_csv("./data/books_summary_final")


In [ ]:
final_df = pd.read_csv("./data/books_summary.csv")

In [ ]:
summary_embeddings = get_synopsis_embeddings(final_df)
np.save('./data/final_embeddings.npy', summary_embeddings)


Saved 1000 embeddings to data/hierarchical_embeddings.npy...


TypeError: object of type 'numpy.float32' has no len()

In [178]:
summaries_embeddings = np.load('./data/embeddingsBackup.npy')

In [100]:
#testing chunking
#tokenize

def test_token_split(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    max_len = 350
    overlap = 50

    chunks = [tokens[i:i + max_len] for i in range(0, len(tokens), max_len - overlap)]
    
    print(f"Total Tokens: {len(tokens)}")
    print(f"Number of Chunks: {len(chunks)}")
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1}: {len(chunk)} tokens")

test_text = "It is written in the form of two parallel lines, with alternating chapters in some ways reflecting each other, and which would converge only late in the course of the book. In the late 15th century Christopher Columbus is working slowly towards the trip across the ocean which would make him famous, and the story follows the many ups and downs of his career and especially his complicated relations with the women in his life. The early chapters, before travelers from the future impinge on his life, can in fact be read as a straightforward historical novel. In a parallel development, in a doomed future where humanity had used up the Earth’s resources and civilization is on the verge of collapse, a team of researchers has developed a machine called the “TruSite II” to view and record the events of the past. (The Concept of a ""Time viewer"" is discussed further on that page). The TruSite II had higher resolution, both visually and temporally, than an earlier competing machine known as the “TempoView,” and an even earlier machine, the “TrueSite I.” Though trying to restore the ravaged earth and preserve personal freedom, the PastWatchers discover that the society in which they live is doomed. The human race is reduced to a population of less than a billion after a century of war, plague, drought, flood and famine. There have been extinctions and the land is poisoned. Ecological damage from misuse of technology cannot be repaired quickly enough to save humanity from even more drastic population reductions, dooming their descendants to primitive, horrific lives for the remainder of the existence of life on Earth. They are endeavoring to record history for future generations, when it is revealed things are far worse than supposed. Upon discovering that the TrueSite II machines (unlike their predecessors) can send information into the past, the scientists set upon a mission to rewrite history and give humanity a second chance for survival. Their efforts are focused on Columbus, a pivotal figure whose religious zealotry and irrational fixation on the New World led to centuries of mass genocide and ecological devastation. When time travel technology is developed, three agents are sent back to the 15th century to alter Columbus’s actions for the better. It’s a one-way trip though, since their timeline along with everyone they know and love will be nullified in the process – an outcome only acceptable since that world is doomed to a fate far worse than extinction anyway. Another sub-story is the discussion about just why Columbus was so obsessed with his westward expedition. A study of his life reveals that the Pastwatch scientists were not the first to realize Columbus’s importance, and that people in a previous incarnation of the timeline had used Columbus’ westward ambition to alter history once before, an action which resulted in our own history. In this “original” history, Columbus never sailed the Atlantic and instead led a last European Crusade to capture Constantinople, taken by the Turks several decades previously. Meanwhile, the Aztec Empire fell and was replaced by an iron-wielding Tlaxcalan empire that was capable of recognizing technological advantages. The Tlaxcala were able to establish a more modern, centralized state and pushed their influence far beyond the old Aztec borders, though still confined to the Americas. This changed, however, when Portuguese sailors in the early 16th century began to be captured by the Tlaxcala and forced to teach them about firearms and large ship construction. This early contact also inflicted the European plagues on the New World empires, which suffered considerable loss of life – but the survivors acquired an immunity. The plagues and arrival of foreigners were taken as a sign that their god, Camaxtli, who demanded human sacrifice, wanted the Tlaxcala to embark on more wars of conquest. Europe of the 16th century was highly politically fragmented to begin with and further weakened by the crusade led by Columbus, and proved to be no match for the invading Tlaxcala armies. Moreover, while the Tlaxcala were already immune to European diseases, the Europeans were vulnerable to such American diseases as syphilis, which accompanied the conquering armies. Later on, the Tlaxcala discovered steam, developed a 19th century type industrial technology while keeping their bloodthirsty, human-sacrificing religion, conquered and enslaved the entire world and destroyed all cultures except for theirs. This history, while dire, eventually developed its own PastWatch which viewed the Tlaxcalan conquest of Europe as the single most tragic event in history. Thus they decided to redirect Columbus’s drive and ambition towards eliminating the fledgling Tlaxcalan empire when it was still vulnerable to European conquest. This action, while technically successful, failed in that they did not go far enough, only trading one flawed timeline for another. The three agents going back in time seek to achieve a balance, and create a timeline where neither Europeans nor Native Americans would cross the ocean on a wave of oppressive conquest. Enormous hard work and endless sacrifice are needed of the three. One of them must deliberately sacrifice his life in destroying Columbus’ ships and stranding him and his crews in the Caribbean. The other man, himself a Central American Indian, works among his own remote ancestors to create a less-aggressive alternative to both the Aztecs and the Tlaxcala. The woman is in the Caribbean, waiting to meet Columbus and his men after they are stranded, to reconcile them with the indigenous tribes and persuade them to share their technology; she eventually becomes Columbus’ lifelong wife and devoted companion. Meanwhile, an artificial germ developed by the genetic engineering of the world they came from is spread through the Americas, designed to give its population immunity to European diseases and thus preventing the decimation they suffered in our timeline upon contact with Europeans. Finally, after decades of hard work, the purpose is achieved: two federations of tribes, one in Central America and the other in the Caribbean, are amalgamated. The structure is held together by the various tribes’ respect for each other, and cemented by the European technology of the Spanish sailors with some additions from the later future, and by a form of tolerant and enlightened Christianity very different from that practiced in contemporary Europe, and especially the Inquisition-ridden Spain. Finally, Columbus returns to Spain more than twenty years after he had set out, at the head of fleet of warships crewed by his new countrymen – making clear that the people of the continent across the ocean can very well defend themselves against the Europeans, but that they come to Europe only for peaceful trade and cultural exchanges – thus averting the evils of both of the previous timelines. Late in his life, Columbus’ future-born wife tells him the truth and reveals the horrific deeds he might have perpetrated and instigated had she not intervened, and he weeps for days. The “redemption” of the title has been achieved. At the end a glimpse is given of this timeline’s Twentieth Century – a harmonious, peaceful and technologically-advanced utopia. Though not explicitly stated, it is implied that this timeline would also avoid the ecological devastation which destroyed its predecessor. Archaeologists in year 1951 (447 of the Humanist Era) disinter an abnormally heavy skull, discover the four indestructible sheets of metal surgically embedded into it by a science clearly more advanced than their own, and follow the indications to recover the three time travelers’ time capsule which will forewarn them of the possible futures."
test_token_split(test_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (1514 > 256). Running this sequence through the model will result in indexing errors


Total Tokens: 1514
Number of Chunks: 6
Chunk 1: 350 tokens
Chunk 2: 350 tokens
Chunk 3: 350 tokens
Chunk 4: 350 tokens
Chunk 5: 314 tokens
Chunk 6: 14 tokens


In [101]:
#test to check if the stacking works correctly
import torch

test_text = "It is written in the form of two parallel lines, with alternating chapters in some ways reflecting each other, and which would converge only late in the course of the book. In the late 15th century Christopher Columbus is working slowly towards the trip across the ocean which would make him famous, and the story follows the many ups and downs of his career and especially his complicated relations with the women in his life. The early chapters, before travelers from the future impinge on his life, can in fact be read as a straightforward historical novel. In a parallel development, in a doomed future where humanity had used up the Earth’s resources and civilization is on the verge of collapse, a team of researchers has developed a machine called the “TruSite II” to view and record the events of the past. (The Concept of a ""Time viewer"" is discussed further on that page). The TruSite II had higher resolution, both visually and temporally, than an earlier competing machine known as the “TempoView,” and an even earlier machine, the “TrueSite I.” Though trying to restore the ravaged earth and preserve personal freedom, the PastWatchers discover that the society in which they live is doomed. The human race is reduced to a population of less than a billion after a century of war, plague, drought, flood and famine. There have been extinctions and the land is poisoned. Ecological damage from misuse of technology cannot be repaired quickly enough to save humanity from even more drastic population reductions, dooming their descendants to primitive, horrific lives for the remainder of the existence of life on Earth. They are endeavoring to record history for future generations, when it is revealed things are far worse than supposed. Upon discovering that the TrueSite II machines (unlike their predecessors) can send information into the past, the scientists set upon a mission to rewrite history and give humanity a second chance for survival. Their efforts are focused on Columbus, a pivotal figure whose religious zealotry and irrational fixation on the New World led to centuries of mass genocide and ecological devastation. When time travel technology is developed, three agents are sent back to the 15th century to alter Columbus’s actions for the better. It’s a one-way trip though, since their timeline along with everyone they know and love will be nullified in the process – an outcome only acceptable since that world is doomed to a fate far worse than extinction anyway. Another sub-story is the discussion about just why Columbus was so obsessed with his westward expedition. A study of his life reveals that the Pastwatch scientists were not the first to realize Columbus’s importance, and that people in a previous incarnation of the timeline had used Columbus’ westward ambition to alter history once before, an action which resulted in our own history. In this “original” history, Columbus never sailed the Atlantic and instead led a last European Crusade to capture Constantinople, taken by the Turks several decades previously. Meanwhile, the Aztec Empire fell and was replaced by an iron-wielding Tlaxcalan empire that was capable of recognizing technological advantages. The Tlaxcala were able to establish a more modern, centralized state and pushed their influence far beyond the old Aztec borders, though still confined to the Americas. This changed, however, when Portuguese sailors in the early 16th century began to be captured by the Tlaxcala and forced to teach them about firearms and large ship construction. This early contact also inflicted the European plagues on the New World empires, which suffered considerable loss of life – but the survivors acquired an immunity. The plagues and arrival of foreigners were taken as a sign that their god, Camaxtli, who demanded human sacrifice, wanted the Tlaxcala to embark on more wars of conquest. Europe of the 16th century was highly politically fragmented to begin with and further weakened by the crusade led by Columbus, and proved to be no match for the invading Tlaxcala armies. Moreover, while the Tlaxcala were already immune to European diseases, the Europeans were vulnerable to such American diseases as syphilis, which accompanied the conquering armies. Later on, the Tlaxcala discovered steam, developed a 19th century type industrial technology while keeping their bloodthirsty, human-sacrificing religion, conquered and enslaved the entire world and destroyed all cultures except for theirs. This history, while dire, eventually developed its own PastWatch which viewed the Tlaxcalan conquest of Europe as the single most tragic event in history. Thus they decided to redirect Columbus’s drive and ambition towards eliminating the fledgling Tlaxcalan empire when it was still vulnerable to European conquest. This action, while technically successful, failed in that they did not go far enough, only trading one flawed timeline for another. The three agents going back in time seek to achieve a balance, and create a timeline where neither Europeans nor Native Americans would cross the ocean on a wave of oppressive conquest. Enormous hard work and endless sacrifice are needed of the three. One of them must deliberately sacrifice his life in destroying Columbus’ ships and stranding him and his crews in the Caribbean. The other man, himself a Central American Indian, works among his own remote ancestors to create a less-aggressive alternative to both the Aztecs and the Tlaxcala. The woman is in the Caribbean, waiting to meet Columbus and his men after they are stranded, to reconcile them with the indigenous tribes and persuade them to share their technology; she eventually becomes Columbus’ lifelong wife and devoted companion. Meanwhile, an artificial germ developed by the genetic engineering of the world they came from is spread through the Americas, designed to give its population immunity to European diseases and thus preventing the decimation they suffered in our timeline upon contact with Europeans. Finally, after decades of hard work, the purpose is achieved: two federations of tribes, one in Central America and the other in the Caribbean, are amalgamated. The structure is held together by the various tribes’ respect for each other, and cemented by the European technology of the Spanish sailors with some additions from the later future, and by a form of tolerant and enlightened Christianity very different from that practiced in contemporary Europe, and especially the Inquisition-ridden Spain. Finally, Columbus returns to Spain more than twenty years after he had set out, at the head of fleet of warships crewed by his new countrymen – making clear that the people of the continent across the ocean can very well defend themselves against the Europeans, but that they come to Europe only for peaceful trade and cultural exchanges – thus averting the evils of both of the previous timelines. Late in his life, Columbus’ future-born wife tells him the truth and reveals the horrific deeds he might have perpetrated and instigated had she not intervened, and he weeps for days. The “redemption” of the title has been achieved. At the end a glimpse is given of this timeline’s Twentieth Century – a harmonious, peaceful and technologically-advanced utopia. Though not explicitly stated, it is implied that this timeline would also avoid the ecological devastation which destroyed its predecessor. Archaeologists in year 1951 (447 of the Humanist Era) disinter an abnormally heavy skull, discover the four indestructible sheets of metal surgically embedded into it by a science clearly more advanced than their own, and follow the indications to recover the three time travelers’ time capsule which will forewarn them of the possible futures."
embedding = get_embedding(test_text)

print("Embedding Shape:", embedding.shape)  # Should be (384,)
print("Embedding Type:", type(embedding))  # Should be a tensor

Embedding Shape: torch.Size([384])
Embedding Type: <class 'torch.Tensor'>


In [180]:
#compute similarities and print top n results
user_embedding = get_embedding("Asterix and Obelix go on an adventure to earn money in Rome")
similarity_scores = util.pytorch_cos_sim(user_embedding, summaries_embeddings)
similarity_scores = similarity_scores.cpu().numpy().flatten()

top_10_indices = np.argsort(similarity_scores)[::-1][:10]  # Sort & get last 10 in reverse

# Print the book titles and their similarity scores
for i, idx in enumerate(top_10_indices):
    print(f"Rank {i+1}: {merged_df['book_name'][idx]} Summary: {merged_df['summaries'][idx]} (Score: {similarity_scores[idx]:.4f})")
print(summaries_embeddings.shape)

Rank 1: A Stranger in the Mirror Summary:  A Stranger in the Mirror The love story between Josephine Czinski (Jill Castle) and Toby Temple is told through a flashback. The Templehaus are German immigrants coming to the USA. Freida Templehaus is the plump wife of a poet who would rather write poetry than work in the meat shop. Angered and faced with reality that her husband is weak, she controls the meat shop and rules the family home. She decides to have a child and they have a son named Tobias, who is noted to be born with an extremely large member. Throughout his childhood, Toby has always strived to earn his busy mother's affections, and has learned to gain it through comedy. However, in his teens, Toby has gained a large libido and after sleeping with many women, impregnates a classmate. To avoid her son marrying what she calls "a nobody", she packs up his bags and sends him to the city to become famous, hoping he would send for her soon. She lies to Eileen's father, the chief of p